## Extending Agents with MCP Servers

In the previous notebook, you learned how to create agents with custom function tools. But what if you want to connect your agent to external services, databases, or APIs without writing all the tool code yourself?

This is where **Model Context Protocol (MCP)** comes in.

### What is MCP?

**Model Context Protocol (MCP)** is an open standard that provides a universal way to connect AI agents to external tools and data sources. Think of it as a "USB-C for AI" - a standardized interface that lets any compatible agent talk to any compatible tool server.

**Why use MCP?**

| Benefit | Description |
|---------|-------------|
| **Standardized** | One protocol works with many different tool providers |
| **Dynamic Discovery** | Agents discover available tools at runtime - no hardcoding |
| **Separation of Concerns** | Tool logic lives in the MCP server, agent logic stays clean |
| **Reusable** | Same MCP server can be used by different agents and applications |

**How it works:**

```
┌─────────────┐      MCP Protocol      ┌─────────────┐
│   Agent     │ ◄────────────────────► │ MCP Server  │
│             │   1. Discover tools    │             │
│  (Client)   │   2. Call tools        │  (Tools)    │
│             │   3. Get results       │             │
└─────────────┘                        └─────────────┘
```

The Microsoft Agent Framework provides `MCPStreamableHTTPTool` to connect to MCP servers over HTTP.

### Setup

First, let's validate our environment and create the Azure OpenAI client.

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Required environment variables
required_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_API_VERSION",
    "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"
]

# Validate setup
missing = [var for var in required_vars if not os.getenv(var)]

if missing:
    print("Missing required environment variables:")
    for var in missing:
        print(f"  - {var}")
    print("\nPlease create a .env file with these variables.")
else:
    print("Setup validated! All required environment variables are set.")

Setup validated! All required environment variables are set.


In [2]:
import os
from agent_framework.azure import AzureOpenAIChatClient

client = AzureOpenAIChatClient(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
)
print("Client created successfully!")

Client created successfully!


---
### Connecting to a Remote MCP Server

Let's start by connecting to a public MCP server: **Microsoft Learn**. This server provides tools to search Microsoft's documentation.

The pattern for using MCP servers is:
1. Create the MCP tool connection using `async with` (ensures proper cleanup)
2. Pass the MCP tool to your agent's `run()` method
3. The agent automatically discovers and uses available tools

In [4]:
from agent_framework import MCPStreamableHTTPTool

# Create an agent for documentation queries
docs_agent = client.create_agent(
    name="DocsAgent",
    instructions="You are a helpful assistant that answers questions using Microsoft documentation."
)

# Connect to Microsoft Learn MCP server
async with MCPStreamableHTTPTool(
    name="Microsoft Learn",
    url="https://learn.microsoft.com/api/mcp",
    chat_client=client
) as mcp_tool:
    
    # The agent can now use tools from the MCP server
    result = await docs_agent.run(
        "How do I create an Azure Storage account using the Azure CLI?",
        tools=mcp_tool
    )
    print(result.text)

To create an Azure Storage account using the Azure CLI, follow these steps:

1. **Sign in to Azure**
   If you're not already signed in, run:
   ```
   az login
   ```

2. **Create a Resource Group (if needed)**
   ```
   az group create --name <resource-group-name> --location <location>
   ```
   Example:
   ```
   az group create --name storage-resource-group --location eastus
   ```

3. **Create the Storage Account**
   Use the `az storage account create` command. Replace the placeholder values with your desired values.
   ```
   az storage account create \
     --name <account-name> \
     --resource-group <resource-group-name> \
     --location <location> \
     --sku Standard_RAGRS \
     --kind StorageV2 \
     --min-tls-version TLS1_2 \
     --allow-blob-public-access false
   ```
   Example:
   ```
   az storage account create \
     --name mystorageaccount123 \
     --resource-group storage-resource-group \
     --location eastus \
     --sku Standard_RAGRS \
     --kind Stor

---
### Running a Local MCP Server

Remote MCP servers are great, but you can also run your own! This workshop includes a **Contoso Customer Service** MCP server that simulates a customer database.

**The server provides 5 tools:**

| Tool | Description |
|------|-------------|
| `list_customers` | List all customers in the system |
| `get_customer` | Get details for a specific customer by ID |
| `get_billing_summary` | Check what a customer owes |
| `get_support_tickets` | View a customer's support tickets |
| `create_support_ticket` | Create a new support ticket |

**To start the server**, open a terminal and run:

```bash
cd mcp
uv run --prerelease=allow mcp_server.py
```

You should see output like:
```
============================================================
Contoso Customer Service MCP Server
============================================================

Available tools:
  - list_customers: List all customers
  - get_customer: Get a specific customer by ID
  - get_billing_summary: Check what a customer owes
  - get_support_tickets: View customer's support tickets
  - create_support_ticket: Create a new support ticket

============================================================

INFO:     Uvicorn running on http://127.0.0.1:8000
```

### Discovering Available Tools

One of MCP's key features is **dynamic tool discovery**. Let's connect to our local server and see what tools are available:

In [7]:
from agent_framework import MCPStreamableHTTPTool

async with MCPStreamableHTTPTool(
    name="Contoso",
    url="http://127.0.0.1:8000/mcp",
    chat_client=client
) as mcp_tool:
    
    print(f"Connected to: {mcp_tool.name}")
    print(f"\nDiscovered {len(mcp_tool.functions)} tools:\n")
    
    for func in mcp_tool.functions:
        print(f"  - {func.name}")
        print(f"    {func.description}\n")

Connected to: Contoso

Discovered 5 tools:

  - list_customers
    List all customers in the system

  - get_customer
    Get detailed information for a specific customer by their ID

  - get_billing_summary
    Get billing summary showing what a customer owes

  - get_support_tickets
    Get support tickets for a customer, optionally filtered to open tickets only

  - create_support_ticket
    Create a new support ticket for a customer



---
### Exercise 1: Simple Customer Lookup

Now let's create an agent that uses the local MCP server. Your task:

1. Connect to the Contoso MCP server at `http://127.0.0.1:8000/mcp`
2. Create an agent with customer service instructions
3. Ask the agent to look up customer ID 2

Fill in the TODOs below:

In [ ]:
from agent_framework import MCPStreamableHTTPTool

# TODO: Create a customer service agent using client.create_agent()
# Give it a name and instructions for helping customers
agent = client.create_agent(
    name="...",
    instructions="..."
)

# TODO: Connect to the local MCP server and run a query
async with MCPStreamableHTTPTool(
    name="Contoso",
    url="...",  # TODO: Add the correct URL
    chat_client=client
) as mcp_tool:
    
    # TODO: Use agent.run() to ask about customer ID 2
    # Remember to pass tools=mcp_tool
    result = await agent.run(
        "...",  # TODO: Your query here
        tools=mcp_tool
    )
    print(result.text)

<details>
<summary>See the solution</summary>

```python
from agent_framework import MCPStreamableHTTPTool

# Create a customer service agent
agent = client.create_agent(
    name="CustomerServiceAgent",
    instructions="You are a helpful customer service agent. Use the available tools to look up customer information and assist with their needs."
)

# Connect to the local MCP server
async with MCPStreamableHTTPTool(
    name="Contoso",
    url="http://127.0.0.1:8000/mcp",
    chat_client=client
) as mcp_tool:
    
    result = await agent.run(
        "Look up the details for customer ID 2",
        tools=mcp_tool
    )
    print(result.text)
```
</details>

---
### Exercise 2: Multi-Step Customer Service

Now let's build a more realistic customer service scenario. Your agent should:

1. Look up a customer's information
2. Check their billing summary
3. Create a support ticket if they have an issue

Use a **multi-turn conversation** (with `AgentThread`) so the agent remembers context between messages.

Fill in the TODOs:

In [ ]:
from agent_framework import MCPStreamableHTTPTool

# Create the agent
agent = client.create_agent(
    name="CustomerServiceAgent",
    instructions="""You are a customer service agent for Contoso. 
    Use the available tools to help customers with their accounts.
    Always be polite and helpful."""
)

# TODO: Create a thread for multi-turn conversation
thread = ...

async with MCPStreamableHTTPTool(
    name="Contoso",
    url="http://127.0.0.1:8000/mcp",
    chat_client=client
) as mcp_tool:
    
    # Conversation messages
    messages = [
        "Hi, I'm customer #1. Can you look up my account?",
        "What's my current billing status?",
        "I'd like to report an issue - my internet has been slow lately. Can you create a ticket for me?"
    ]
    
    for message in messages:
        print(f"\n{'='*50}")
        print(f"Customer: {message}")
        print(f"{'='*50}")
        
        # TODO: Run the agent with the message, thread, and tools
        result = await agent.run(
            ...,  # TODO: pass the message
            thread=...,  # TODO: pass the thread
            tools=...  # TODO: pass the MCP tool
        )
        print(f"\nAgent: {result.text}")

<details>
<summary>See the solution</summary>

```python
from agent_framework import MCPStreamableHTTPTool

# Create the agent
agent = client.create_agent(
    name="CustomerServiceAgent",
    instructions="""You are a customer service agent for Contoso. 
    Use the available tools to help customers with their accounts.
    Always be polite and helpful."""
)

# Create a thread for multi-turn conversation
thread = agent.get_new_thread()

async with MCPStreamableHTTPTool(
    name="Contoso",
    url="http://127.0.0.1:8000/mcp",
    chat_client=client
) as mcp_tool:
    
    messages = [
        "Hi, I'm customer #1. Can you look up my account?",
        "What's my current billing status?",
        "I'd like to report an issue - my internet has been slow lately. Can you create a ticket for me?"
    ]
    
    for message in messages:
        print(f"\n{'='*50}")
        print(f"Customer: {message}")
        print(f"{'='*50}")
        
        result = await agent.run(
            message,
            thread=thread,
            tools=mcp_tool
        )
        print(f"\nAgent: {result.text}")
```
</details>

---
## Summary & Recap

In this notebook, you learned how to extend your agents with **Model Context Protocol (MCP)** servers:

### Key Concepts

| Concept | Description |
|---------|-------------|
| **MCP (Model Context Protocol)** | A standard protocol for connecting AI agents to external tools and services |
| **MCPStreamableHTTPTool** | Agent Framework class for connecting to MCP servers over HTTP |
| **Tool Discovery** | MCP servers expose their tools dynamically - agents discover them at runtime |
| **Remote vs Local** | MCP servers can be hosted services or run locally |

### What You Built

1. **Documentation Agent** - Connected to Microsoft Learn MCP for real-time docs lookup
2. **Tool Discovery** - Inspected available tools from an MCP server
3. **Customer Service Agent** - Used a local MCP server for multi-turn customer support

### Key Pattern

```python
# Connect to MCP server
async with MCPStreamableHTTPTool(
    name="ServerName",
    url="http://server-address/mcp",
    chat_client=client
) as mcp_tool:
    
    # Use with your agent
    result = await agent.run(
        "Your query",
        tools=mcp_tool
    )
```

### Chapter 1 Complete!

You now have the foundation for building single agents with:
- Custom function tools (01.1)
- Multi-turn conversations (01.1)
- Middleware for observability (01.1)
- MCP integration for external services (01.2)

In **Chapter 2**, you'll learn how to orchestrate **multiple agents** working together in workflows!